# Synchronize Image, Depth, and Pose data

This notebook synchronizes data from extracted bagfiles. Assuming the data directory containt are like so:
> ```
dataDir
    rgb\ # containes rgb images
        0.00000.png
        0.00001.png
        ...
    depth\ # containes depth images
        0.00000.png
        0.00001.png
        ...
    rgb.txt # index of rgb files by time
    depth.txt # index of depth files by time
    tf.txt # index of pose files by time
```

The resulting output is like so:

> ```
dataDir
    images\ # containes synchronize rgb depth image pairs
        0.00001-rgb.png
        0.00001-depth.png
        0.00002-rgb.png
        0.00002-depth.png
        ...
        pose.txt # respective poses indexed by time
```

In [1]:
import errno
import glob
import numpy as np
import os
import pandas as pd
import shutil
from rgbd_benchmark_tools import associate

## Configure parameters

In [2]:
dataDir = '/home/ruffsl/Data/objects/optical4.ojjuice'
sample_rate = 1 # output sample rate (every x image pair)
frame_rate = 30 # input frame rate (fps)
tolerance = 0.6 # 60% period tolerance of frame rate
rgb_to_tf_offset = 0 # in seconds
rgb_to_depth_offset = 0 # in seconds

## Define helper functions

In [3]:
def mkdir_p(path):
    try:
        os.makedirs(path)
    except OSError as exc:  # Python >2.5
        if exc.errno == errno.EEXIST and os.path.isdir(path):
            pass
        else:
            raise

def insertionSort(alist):
    for index in range(1,len(alist)):
        currentvalue = alist[index]
        position = index
        while position>0 and alist[position-1][0]>currentvalue[0]:
            alist[position]=alist[position-1]
            position = position-1
        alist[position]=currentvalue
    return alist

def cp_file(src, dst, time, label):
    for file in glob.glob(src+'*'):
        extension = os.path.splitext(file)[1]
        file2 = os.path.join(*[dst,time+label+extension])
        shutil.copy2(file, file2)

## Create paths

In [4]:
image_cleaned = os.path.join(dataDir,'images')
pose_file = os.path.join(image_cleaned,'pose.txt')
mkdir_p(image_cleaned)

tf_file = os.path.join(dataDir,'tf.txt')
rgb_file = os.path.join(dataDir,'rgb.txt')
depth_file = os.path.join(dataDir,'depth.txt')

## Load and associate data

In [5]:
tf_list = associate.read_file_list(tf_file)
rgb_list = associate.read_file_list(rgb_file)
depth_list = associate.read_file_list(depth_file)

max_difference = tolerance / frame_rate
rgb_tf_matches = associate.associate(rgb_list, tf_list, rgb_to_tf_offset, max_difference)
rgb_depth_matches = associate.associate(rgb_list, depth_list, rgb_to_depth_offset, max_difference)

## Sort data by time

In [6]:
rgb_depth_matches = insertionSort(rgb_depth_matches)

rgb_matches = np.array(rgb_depth_matches)[:,0]
depth_matches = np.array(rgb_depth_matches)[:,1]

## Load pose info into dataframe

In [7]:
pose_df = pd.read_csv(tf_file, header=None, delim_whitespace=True, dtype={0:str})
pose_df = pose_df.set_index(0)

## Loop over matches and copy files

In [8]:
poses = []
for rgb_image, tf_pose in rgb_tf_matches[0::sample_rate]:
    rgb_match = np.searchsorted(rgb_matches, rgb_image)
    if rgb_image == rgb_matches[rgb_match]:
        rgb_image = os.path.join(*[dataDir,'rgb',"{:.6f}".format(rgb_image)])
        depth_image = depth_matches[rgb_match]
        depth_image = os.path.join(*[dataDir,'depth',"{:.6f}".format(depth_image)])
        time = "{:.6f}".format(tf_pose)
        cp_file(rgb_image,image_cleaned,time,'-rgb')
        cp_file(depth_image,image_cleaned,time,'-depth')
        poses.append(time)

## Save pose file as well

In [9]:
pose_df2 = pose_df[pose_df.index.map(lambda x: x in poses)]
pose_df2.to_csv(pose_file, header=None)